## Model Train 1 - Tesis Javier-Uriel

### Importamos algunas librerías que nos serán útiles más adelante

In [29]:
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers
assert (tf.__version__=='2.4.1'), 'Versión incorrecta de Tensorflow, por favor instale 2.4.1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from IPython.display import clear_output

pd.set_option('display.max_columns', None) #Para mostrar todas las columnas

import gc #garbage collector
import gc; gc.enable()

Num GPUs Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Leemos el Dataset

In [3]:
#Dataset solo movimientos en Z
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3']
states_list_org = ["vz","az", "uvz"]#, 
#                    "p", "q",
#                    "wp", "wq", 
#                    "ap", "aq"]
dataset_name = "/Dataset_Z3"
directory = "../logs/Datasets"
ORDER = 3
states_list=states_list_org.copy()

In [4]:
def pandas_read():
    dfs = []
    global states_list
    # reading train data
    for filename in os.listdir(directory+dataset_name):
        if not filename.endswith(".csv"):
            continue
        df = pd.read_csv(os.path.join(directory+dataset_name, filename))
        a = []
        ## Desplazamos estados anteriores        
        for n in range(1,ORDER+1):
            for column in states_list:
                df[column+str(n)] = df[column].shift(periods=n, fill_value=0)
                a.append(column+str(n))
        dfs.append(df)
    states_list+=a       

    return pd.concat(dfs)

In [5]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    dataset = pandas_read()
else:
    for column in states_list_org:
        for n in range(1,ORDER+1):
            states_list.append(column+str(n))

### Estados repetidos

En este caso se eliminan estados repetidos y estados que se encuentren en estado transitorio mientras el dron despega o se estabiliza antes de introducir la señal de control.

In [6]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    shape_b4 = dataset.shape
    dataset = dataset.drop(["timestamps"], axis=1).drop_duplicates()
    print(f'len (b4 drop) - len = {shape_b4[0]-dataset.shape[0]}')

### División del dataset en estados y acciones

In [7]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    actions = dataset[rpm_list]
    actions.describe()

#### Normalización de acciones

In [8]:
def normalize_df(df):
    K = df.max().max()
    df_norm = actions/K
    return df_norm, {'K':K}

In [9]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    actions, K = normalize_df(actions)
    print(K)
    dataset = dataset.drop(columns=rpm_list)
    dataset = pd.concat([dataset, actions], axis=1)

#### Definimos los estados

In [10]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    states = dataset[states_list]
    print(f'columns = {states.columns}')
    print(f'shape = {states.shape}')
    #states.head()

#### Guardamos el dataset en un nuevo archivo .csv

In [11]:
if not os.path.isfile(directory+"_train"+dataset_name+".csv"):
    dataset.to_csv(directory+"_train"+dataset_name+".csv", index=False)
    del dataset
    del states, actions

#### Creamos dataset de Tensorflow a partir de csv
Esto es con el fin de no cargar todo el dataset.

In [55]:
BATCH_SIZE = 10000
dataset = tf.data.experimental.make_csv_dataset(directory+"_train"+dataset_name+".csv", 
                                                select_columns = states_list + rpm_list,
                                                batch_size = BATCH_SIZE,
                                                num_epochs=1,
                                                shuffle=True)

def make_features_labels_tuple(x):
    labels = {k: x[k] for k in x if k in rpm_list}
    features = {k: x[k] for k in x if k in states_list}
    return (features, labels)

dataset = dataset.map(lambda x: make_features_labels_tuple(x))

In [36]:
dataset = list(dataset.as_numpy_iterator())

### Dividimos el dataset

In [40]:
batch_train, batch_test = train_test_split(dataset, test_size=0.1)

In [44]:
next(iter(batch_train))

({'vz': array([-0.8156686 , -0.12675412, -4.0878735 , ...,  4.0946016 ,
         -2.3261452 , -1.8058904 ], dtype=float32),
  'az': array([-5.5710807e+00,  7.1809016e-02, -1.6574329e-01, ...,
          0.0000000e+00, -5.3234096e+00, -3.9405812e-09], dtype=float32),
  'uvz': array([-2.5673592 , -0.12228362, -4.13966   , ...,  4.13966   ,
         -5.5821524 , -1.8168355 ], dtype=float32),
  'vz1': array([-0.79245573, -0.12705332, -4.087183  , ...,  4.0946016 ,
         -2.3039641 , -1.8058904 ], dtype=float32),
  'vz2': array([-0.769233 , -0.1273526, -4.0864916, ...,  4.0946016, -2.2817626,
         -1.8058904], dtype=float32),
  'vz3': array([-0.7460005 , -0.12765154, -4.0857987 , ...,  4.0946016 ,
         -2.2595406 , -1.8058904 ], dtype=float32),
  'az1': array([-5.5734601e+00,  7.1824037e-02, -1.6599709e-01, ...,
          0.0000000e+00, -5.3283701e+00, -3.9436188e-09], dtype=float32),
  'az2': array([-5.5757976e+00,  7.1748510e-02, -1.6625124e-01, ...,
          0.0000000e+00, -5.

## Keras Model

## Callbacks

#### Early Stopping

In [45]:
Early_Stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

#### Plotting

In [46]:
class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.yscale('log')
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

#### Definición del Modelo

In [47]:
inputs = tf.keras.Input(shape=(len(states_list),))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(len(rpm_list), activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

#### Compilado el Modelo

In [48]:
metrics = ['mean_squared_error']
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=metrics)

#### Entrenamiento del Modelo

In [53]:
%%time
history = model.fit(dataset[0], epochs=100, callbacks=[Early_Stopping, plot_losses], verbose=0)

ValueError: in user code:

    C:\Users\mrjar\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\mrjar\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\mrjar\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\mrjar\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\mrjar\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\mrjar

<Figure size 432x288 with 0 Axes>

#### Evaluación del Modelo

In [34]:
loss, mean_sq = model.evaluate(X_test, Y_test)
print(f'mean_sq: {mean_sq} -> {mean_sq*K["K"]} RPM')
print(f'loss: {loss} -> {loss*K["K"]} RPM')

54950/54950 [==============================] - 26s 476us/step - loss: 8.7701e-05 - mean_squared_error: 8.7701e-05s - loss: - E - ETA: 0s - loss: 8.7668e-05 - mean_squ - ETA: 0s - loss: 8.7702e-05 - mean_squared_error: 8.7702e-
mean_sq: 8.770081331022084e-05 -> 1.900165067293201 RPM
loss: 8.770081331022084e-05 -> 1.900165067293201 RPM


#### Se guarda el Modelo

In [35]:
I = 0
model.save(f'../Models/{dataset_name}_{I}.h5')

In [36]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_{I}.h5')
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 44        
Total params: 284
Trainable params: 284
Non-trainable params: 0
_________________________________________________________________


In [37]:
%%time
x_test = X_test.sample(n=3, random_state=1)
for index, sample in x_test.iterrows():
    print(model.predict([list(sample)])*K['K'])

[[14443.009  14443.0205 14443.024  14443.014 ]]
[[13763.176 13763.191 13763.189 13763.182]]
[[14339.956 14339.97  14339.971 14339.962]]
Wall time: 206 ms


In [38]:
%%time
x_test = [0]*len(states_list)
x_test[0] = 1
x_test[1] = -9.8
x_test[2] = 3
print(model.predict([list(x_test)])*K['K'])

[[20097.742 20097.686 20097.742 20097.748]]
Wall time: 41.9 ms
